<a href="https://colab.research.google.com/github/kejeon/in_dev_RN20Q/blob/main/6_sweep_resent20_q_with_KD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
IN_COLAB = 'google.colab' in sys.modules
IN_LOCAL = not IN_COLAB
USE_GITHUB = True
USE_DRIVE = False
device = 'cuda'

assert not (USE_GITHUB and USE_DRIVE)

In [2]:
if IN_COLAB:
  !pip install wandb -qU
  from google.colab import runtime
  if USE_GITHUB:
    !git clone https://github.com/kejeon/in_dev_RN20Q.git
    %cd '/content/in_dev_RN20Q'
  elif USE_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
    %cd '/content/drive/MyDrive/GitHub/in_dev_RN20Q'

import wandb
import torch
from model.resnet4c10q import ResNet20_Q
from model.resnet4c10 import resnet20
from mylib.KDResNetTrainer import ResNetTrainer

wandb.login(key='e0c11d3ff2bee4c8775ba05863038fdac671c043')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.7/214.7 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.6 MB/s eta 0:00:00
Cloning into 'in_dev_RN20Q'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 123 (delta 61), reused 77 (delta 28), pack-reused 0
Receiving objects: 100% (123/123), 2.04 MiB | 7.83 MiB/s, done.
Resolving deltas: 100% (61/61), done.
/content/in_dev_RN20Q


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
api = wandb.Api()
artifact = api.artifact('jke1994/ResNet20/model:v179')
artifact.download(root='./pretrained_model')
state_dict = torch.load('./pretrained_model/ckpt.pth', map_location=torch.device(device))

teacher_model = resnet20()

try:
  teacher_model.load_state_dict(state_dict['net'])
except:
  # if the model is wrapped in a module, update all keys in state_dict to remove module.
  state_dict['net'] = {k.replace('module.', ''): v for k, v in state_dict['net'].items()}
  teacher_model.load_state_dict(state_dict['net'])

wandb:   1 of 1 files downloaded.  


In [4]:
def train():
    run = wandb.init()

    arch_tag = "ResNet20_Q"
    batch_size = 128
    dataset = "CIFAR10"
    T=wandb.config.T
    alpha=wandb.config.alpha
    lr=0.012

    student_model = ResNet20_Q(a_bit=4, w_bit=1)

    my_trainer = ResNetTrainer(dataset=dataset,
                           arch_tag=arch_tag,
                           student_model=student_model,
                           teacher_model=teacher_model,
                           T=T,
                           alpha=alpha,
                           device=device,
                           batch_size=batch_size,
                           lr=lr)

    my_trainer.train_script(200)

    return

In [ ]:
wandb.agent(sweep_id='qkr840ok', function=train, count=5, project='my-first-sweep')

wandb: Agent Starting Run: mm6dnou4 with config:
wandb: 	T: 20
wandb: 	alpha: 0.2
wandb: Currently logged in as: jke1994. Use `wandb login --relogin` to force relogin


100%|██████████| 170498071/170498071 [00:02<00:00, 79674779.14it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified

Epoch: 0


Epoch 0:   0%|          | 0/391 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:2919: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(
Epoch 0: 100%|██████████| 391/391 [00:55<00:00,  7.10it/s, train_acc=0.183, xentropy=1.979]
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:437: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


test_acc: 0.241
Saving..

Epoch: 1


Epoch 1: 100%|██████████| 391/391 [00:48<00:00,  7.99it/s, train_acc=0.247, xentropy=1.831]
